# 텍스트 데이터 다루기

텍스트 처리성능에 큰 영향을 주는 다양한 텍스트 데이터 전처리 기법을 소개한다. Word나 형태소 레벨의 tokenizer 및 Subword 레벨 tokenizing 기법(BPE, sentencepiece)을 소개하고 사용법을 익힌다.

- 들어가며
- 전처리 : 자연어의 노이즈 제거
- 분산표현 : 바나나와 사과의 관계를 어떻게 표현할까?
- 토큰화
  - 그녀는? 그녀+는?
  - 다른 방법들
- 토큰에게 의미를 부여하기
- 마무리

## 들어가며

![출처 : Photo by Annie Spratt on Unsplash](img/00.png)

- 자연어(Natural language) : 일상에서 자연히 발생하여 쓰이는 언어
- 인공어(Artificial language) : 자연어의 반대말
  - ex) 프로그래밍 언어(Programming language)

- 문맥 자유 문법(Context-Free Grammar)
- 문맥 의존 문법(Context-Sensitive Grammar)

[자연 언어 와 프로그래밍 언어, AI Study](https://dukeyang.tistory.com/2)

어려운 본질적 이유

프로그래밍 언어에는 파서가 존재
- 문맥과 상황에 따라 그 뜻이 달라지지 않는 언어

[Announcing SyntaxNet, Google AI Blog](https://ai.googleblog.com/2016/05/announcing-syntaxnet-worlds-most.html)

`착한 영희`의 친구 vs 착한 `영희의 친구` 

- 언어의 문법만으로 충분하지 않고 그 언어의 의미까지 이해하는 과정이 필요
- 단어의 의미 단위로 쪼개 내는 작업 자체도 만만치 않다

자연어를 의미 단위로 쪼개는 토큰화(Tokenization) 기법

Wordpiece Model같은 Subword level의 처리 기법

- 분산 표현에 대한 직관적 이해
- 문장 데이터를 정제하는 방법
- 토큰화의 여러 가지 기법들
- 단어 Embedding을 구축하는 방법

## 전처리 : 자연어의 노이즈 제거

![01](img/01.png)

이상적인 데이터의 형태는 이른바 교과서적인 문장들
- 그러나 단어가 출현하게 될 확률 모델을 배움
- 문장은 예외적으로 변형될 여지가 무궁무진(띄어쓰기, 맞춤법, 약어)

노이즈 정리
- 불완전한 문장
  - 끊어서 전송
- 문장의 길이가 너무 길거나 짧은 경우
  - 감정 표현(ㅠㅜ, ㅋㅋ), 의미없이 긴 문장
- 문장 시간 간격이 긴 경우
  - 사람이 많은 카톡방
- 바람직하지 않은 문장
  - 욕설, 오타

한국어를 고려하기에는 너무 높고 영어를 처리해보자!

문장 부호 입력
- Hi, my name is john.

In [1]:
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")

    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is john . 


단순히 문장부호 양옆에 공백을 추가해 주는 방식은 문제를 야기할 수도 있습니다

숫자(12,345), 이름(Mr.Byeongryul) 불가피한 손실로 취급하고 넘어갑니다.

대소문자 입력
- First, open the first chapter.

In [2]:
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


특수문자
- He is a ten-year-old boy.

In [3]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


원하는 문자외에 제거함 [Python 정규표현식(Regex)](https://docs.python.org/ko/3/library/re.html)

In [4]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

## 분산표현 : 바나나와 사과의 관계를 어떻게 표현할까?

- 희소 표현(Sparse representation) 
  - 각 차원에 의미를 부여하여 값을 넣음
  - 차원이 늘어날수록 0 이 늘어남
  - 너무 고차원이 필요하고 불필요한 메모리와 연산량이 낭비된다
  - 의미적 유사도를 계산할 수 없다 [위키독스: 코사인 유사도](https://wikidocs.net/24603)
- 분산 표현(distributed representation) 
  - 각 단어가 몇 차원의 속성을 가질지 정의


## 토큰화

### 그녀는? 그녀+는?

우리가 정의할 토큰화 기법이 결정할 부분
- 1: 그녀는 2: 나와 3: 밥을 4: 먹는다
- 1: 그녀 2: 는 3: 나 4: 와 5: 밥 6: 을 7: 먹는다 

- 공백 기반
  - days 와 day 가 구분되어 따로 저장이 되는 정도의 문제
- 형태소 기반 토큰화
  - 한글은 공백기반이 올지 않음
  - 대표적인 분석기 [KoNLPy: 파이썬 한국어 NLP - KoNLPy 0.4.3 documentation](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)
  

In [7]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
[Komoran] 
[('코로나바이러스', 'NNP'), ('는', 'JX'), ('2019', 'SN'

OOV(Out-Of-Vocabulary) 문제
- 사전에 없으면, 특수한 토큰(Unknown Token)으로 치환
- 새로 등장한(본 적 없는) 단어에 대해 약한 모습

### 다른 방법들

- Byte Pair Encoding(BPE) : 가장 많이 등장하는 바이트 쌍(Byte Pair) 을 새로운 단어로 치환하여 압축하는 작업
- Wordpiece Model(WPM) : 한 단어를 여러 개의 Subword의 집합으로 보는 방법
  - [Neural Machine Translation of Rare Words with Subword Units
](https://arxiv.org/pdf/1508.07909.pdf)


In [ ]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

극단적으로 생각했을 때 알파벳 26개와 특수문자, 문장부호를 아무리 추가해도 100개 이내로 사전을 정의할 수 있음

떤 기준으로 이들을 결합해서 문장을 복원

Wordpiece Model(WPM)

- 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.
- 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다.

[JAPANESE AND KOREAN VOICE SEARCH](https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf)

[확률(probability)과 가능도(likelihood) 그리고 최대우도추정(likelihood maximazation)](https://jjangjjong.tistory.com/41)

![[▲ Katz's Back-off Model]](img/02.png)

[google/sentencepiece](https://github.com/google/sentencepiece)

고성능의 BPE

soynlp

[soynlp](https://github.com/lovit/soynlp)

형태소 기반인 koNLPy의 단점을 해결하기 위해 soynlp를 사용

통계 기반 토크나이저

- 자동`차` vs 보이`차`
- 유사도가 같은지 어떻게 판단하는가?

## 토큰에게 의미를 부여하기

- Word2Vec [위키독스/Word2Vec](https://wikidocs.net/22660)
  - 동시에 등장하는 단어끼리는 연관성 확인
- FastText [한국어를 위한 어휘 임베딩의 개발 -1-](https://brunch.co.kr/@learning/7)
  - 연산의 빈부격차 존재(많이 나온 단어 더 많이 연산)
- ELMo - the 1st Contextualized Word Embedding [전이 학습 기반 NLP (1): ELMo](https://brunch.co.kr/@learning/12)
  - 고정적 의미를 피해감

## 마무리